# Preprocessing The DataFrames

## Imports

In [1]:
import pandas as pd
from googletrans import Translator

## Reading the Dataframes

In [2]:
porsche_insta_df = pd.read_csv("./data/porsche_raw_instagram_dataset.csv")
bmw_insta_df = pd.read_csv("./data/bmw_raw_instagram_dataset.csv")
mercedes_insta_df = pd.read_csv("./data/mercedes_raw_instagram_dataset.csv")
porsche_lepzig_df = pd.read_csv("./data/porsche_leipzig_raw_instagram_dataset.csv")

In [3]:
print(porsche_insta_df.shape)
print(bmw_insta_df.shape)
print(mercedes_insta_df.shape)
print(porsche_lepzig_df.shape)

(1268, 54)
(1000, 30)
(1000, 58)
(1000, 62)


## Preprocessing pipeline

In [7]:
def drop_unecessary_columns(df: pd.DataFrame):
    
    column_list = [
        'location','locationId', "isSidecar", "query",
         "videoUrl", "timestamp",'likedByViewer', "imgUrl",
        'fullName', 'videoDuration', "caption", "postId"]
    df = df.drop(columns=column_list, axis=1)
    df = df.loc[:,~df.columns.str.startswith("tagged")]
    return df

def rename_columns(df: pd.DataFrame):
    rename_cols = {
        "description": "postContent",
        "pubDate": "postTimestamp"
    }
    return df.rename(columns=rename_cols)

def change_datatypes(df: pd.DataFrame):
    df["postTimestamp"] = pd.to_datetime(df["postTimestamp"])
    return df

def get_current_year_data(df: pd.DataFrame, year: int):
    return df[df["postTimestamp"] >= f"{year}.01.01"].sort_values("postTimestamp")

def remove_duplicates(df: pd.DataFrame):
    unique_data_size = len(df.postUrl.unique())
    dataset_size = len(df)
    if unique_data_size != dataset_size:
        return df.drop_duplicates(subset=["postUrl"])
    else:
        return df

In [8]:
def pipeline(df: pd.DataFrame):
    df = drop_unecessary_columns(df=df)
    df = rename_columns(df=df)
    df = change_datatypes(df=df)
    df = get_current_year_data(df=df, year=2023)
    df = remove_duplicates(df=df)
    return df

In [9]:
porsche_df = pipeline(df=porsche_insta_df)
bmw_df = pipeline(df=bmw_insta_df)
mercedes_df = pipeline(df=mercedes_insta_df)
porsche_leipzig_df = pipeline(df=porsche_lepzig_df)

In [11]:
print(porsche_df.shape)
print(bmw_df.shape)
print(mercedes_df.shape)
print(porsche_leipzig_df.shape)

(129, 10)
(138, 10)
(205, 10)
(153, 10)


## Splitting the text in Dataset(Porsche PEC)

The post content consists of German and English translations and we split the text to extract only the German version.

In [14]:
porsche_df.head()

,postUrl,postContent,commentCount,likeCount,postTimestamp,type,profileUrl,username,viewCount,playCount
414,https://www.instagram.com/p/Cm6QO3Uoz-A/,Das neue Jahr hat begonnen und wir können es s...,2,298,2023-01-02 10:56:14+00:00,Photo,https://www.instagram.com/pec.hhr,pec.hhr,NaN,NaN
411,https://www.instagram.com/p/CnAKVbkNMQG/,"Eines der großen Highlights, die 2023 auf Euch...",1,210,2023-01-04 18:00:09+00:00,Photo,https://www.instagram.com/pec.hhr,pec.hhr,NaN,NaN
409,https://www.instagram.com/p/CnE28ZQISr0/,"Mit frischer Energie, jeder Menge neuer Ideen ...",0,149,2023-01-06 13:46:55+00:00,Photo,https://www.instagram.com/pec.hhr,pec.hhr,NaN,NaN
407,https://www.instagram.com/p/CnM9OMnIjOS/,"""Der 911 ist das einzige Auto, mit dem man von...",1,410,2023-01-09 17:15:41+00:00,Photo,https://www.instagram.com/pec.hhr,pec.hhr,NaN,NaN
406,https://www.instagram.com/p/CnSGXu7ItaI/,Weil echte Sportler bei jedem Wetter in Bewegu...,2,383,2023-01-11 17:16:16+00:00,Video,https://www.instagram.com/pec.hhr,pec.hhr,NaN,7122.0


In [15]:
filtered_content_en = []
filtered_content_de = []
translator = Translator()

for i in range(len(porsche_df)):
    text = porsche_df.postContent.values[i].split("\n")[0]
    filtered_content_de.append(text)
    text = translator.translate(text).text
    filtered_content_en.append(text)

In [16]:
porsche_df["filteredContentEn"] = filtered_content_en
porsche_df["filteredContentDe"] = filtered_content_de

## Translating Porsche Leipzig posts

In [17]:
def translate_text(text):
    translator = Translator()
    return translator.translate(text).text

In [18]:
porsche_leipzig_df["postContentEn"] = porsche_leipzig_df["postContent"].apply(translate_text)

## Saving preprocessed DF

In [20]:
porsche_df.to_csv("./data/clean_datasets/porsche_pec_instagram_clean.csv", index=False)
bmw_df.to_csv("./data/clean_datasets/bmw_instagram_clean.csv", index=False)
mercedes_df.to_csv("./data/clean_datasets/mercedes_instagram_clean.csv", index=False)
porsche_leipzig_df.to_csv("./data/clean_datasets/porsche_leipzig_instagram_clean.csv", index=False)

/Users/kaushikdayalan/projects/storymachine_projects/porsche_topic_analysis/.env/lib/python3.9/site-packages/pandas/core/internals/blocks.py:2540: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
